In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

if not torch.backends.mps.is_available():
    device = torch.device('cpu')
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")

else:
    device = torch.device("mps")

# Basic block for ResNet
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

# ResNet model
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 32

        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1, bias=False)  # 1 input channel
        self.bn1 = nn.BatchNorm2d(32)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[1], stride=2)
        self.linear = nn.Linear(17920 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        # x = self.layer4(x)
        x = F.avg_pool2d(x, 4)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x

def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


In [5]:
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split

resize_scale = 0.5

def custom_transform(image):
    # Crop the image to the desired region of interest (ROI)
    image = image.crop((55, 35, 390, 253))
    # Convert the cropped image to a PyTorch tensor
    return transforms.ToTensor()(image)

def load_data(resize_images = False):
    # Define your data transformation (without resizing)
    transforms_list = []
    transforms_list.append(transforms.Grayscale(num_output_channels=1))
    transforms_list.append(transforms.Lambda(custom_transform))  # Apply the custom transformation
    if resize_images:
        transforms_list.append(transforms.Resize((int(218*resize_scale), int(335*resize_scale))))
    transforms_list.append(transforms.Normalize((0.5,), (0.5,)))  # Normalize to [-1, 1]
    data_transform = transforms.Compose(transforms_list)

    class CustomImageDataset(datasets.ImageFolder):
        def __init__(self, root, transform=None):
            super(CustomImageDataset, self).__init__(root=root, transform=transform)

    # Define the path to your data folder
    data_dir = 'data/images_original'

    # Create an instance of your custom dataset
    custom_dataset = CustomImageDataset(root=data_dir, transform=data_transform)

    # Calculate the size of the training and testing sets
    total_size = len(custom_dataset)
    train_size = int(0.8 * total_size)
    test_size = total_size - train_size

    # Split the dataset
    train_dataset, test_dataset = random_split(custom_dataset, [train_size, test_size])
    batch_size = 64  # You can adjust this batch size as needed

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

train_loader, test_loader = load_data(resize_images=True)

# Get a batch of data from the training loader
data_iterator = iter(train_loader)
images, labels = next(data_iterator)

print("Image shape:", images[0].shape)
num_channels = images[0].shape[0]  # The number of channels in the image
print("Number of channels:", num_channels)

Image shape: torch.Size([1, 109, 167])
Number of channels: 1


In [6]:
import torch.nn as nn
import torch.optim as optim

# Define your neural network
net = ResNet18()
net.to(device)

# Define the loss function (criterion) and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification
optimizer = optim.SGD(net.parameters(), lr=0.0005, momentum=0.9)

# Set the number of training epochs
num_epochs = 20

# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0

    # Iterate over the training dataset
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = net(inputs)

        # Compute the loss
        loss = criterion(outputs, labels)

        # Backpropagation and optimization
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()

    # Evaluate the model on the test dataset and calculate accuracy
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_accuracy = 100 * correct / total
    print(f"Epoch {epoch + 1} | Loss: {running_loss:.3f} | Accuracy: {test_accuracy:.2f}%")
    running_loss = 0.0

print("Training finished")

# Save the trained model if desired
# torch.save(net.state_dict(), "my_model.pth")


Epoch 1 | Loss: 39.693 | Accuracy: 20.50%
Epoch 2 | Loss: 32.555 | Accuracy: 29.50%
Epoch 3 | Loss: 25.801 | Accuracy: 42.50%
Epoch 4 | Loss: 17.750 | Accuracy: 46.50%
Epoch 5 | Loss: 12.179 | Accuracy: 58.00%
Epoch 6 | Loss: 8.784 | Accuracy: 53.00%
Epoch 7 | Loss: 7.376 | Accuracy: 65.50%
Epoch 8 | Loss: 5.573 | Accuracy: 57.50%
Epoch 9 | Loss: 4.812 | Accuracy: 57.50%
Epoch 10 | Loss: 3.982 | Accuracy: 62.50%
Epoch 11 | Loss: 2.495 | Accuracy: 65.00%
Epoch 12 | Loss: 1.960 | Accuracy: 65.50%
Epoch 13 | Loss: 1.368 | Accuracy: 63.00%
Epoch 14 | Loss: 1.177 | Accuracy: 69.00%
Epoch 15 | Loss: 0.924 | Accuracy: 66.50%
Epoch 16 | Loss: 0.800 | Accuracy: 67.00%
Epoch 17 | Loss: 0.617 | Accuracy: 69.00%
Epoch 18 | Loss: 0.535 | Accuracy: 68.50%
Epoch 19 | Loss: 0.580 | Accuracy: 68.50%
Epoch 20 | Loss: 0.468 | Accuracy: 70.50%
Training finished
